In [11]:
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.stats import ttest_ind
import scipy.stats as stats
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("query_results.csv")

In [3]:
df.head()

,id,group,total_spent_per_user
0,1014313,B,0.0
1,1029532,B,0.0
2,1018168,A,0.0
3,1029599,A,0.0
4,1025920,B,0.0


# $H_0$: The control and treatment group's average spent will remain the same. 

In [33]:
def get_mean_and_se(df, col):

    mean = df[col].mean()
    se = df[col].std()/np.sqrt(len(df))
    return mean, se

In [34]:
df_a = df[df["group"]=="A"]

df_b = df[df["group"]=="B"]

mean_a, std_a = get_mean_and_se(df_a, "total_spent_per_user")

mean_b, std_b = get_mean_and_se(df_b, "total_spent_per_user")

sig = 0.05


In [35]:
def critical_value(sig, dof, test_kind='two-tail'):
    '''Calculate critical value for various tests.

    Parameters:
    -----------
    sig: float
        Significance level between 0 and 1 e.g. 0.05
    dof: int
        Degrees of freedom i.e. number of data points - 1
    test_kind: string
        Type of test i.e. left, right, two-tail. 

    Returns: 
    --------
    critical_value: float 
        The critical value.'''

    if test_kind == 'left':
        crit = t.ppf(q=sig, df=dof)
    elif test_kind == 'right':
        crit = t.ppf(q=1-sig, df=dof)
    else:
        crit = t.ppf(q=1-sig/2, df=dof)
    return crit

In [36]:
critical_a = critical_value(sig, dof=len(df_a) - 1)
critical_b = critical_value(sig, dof=len(df_b) - 1)

In [37]:
def confidence_interval(mean, se, critical_value):
    ''' '''

    lower = mean - critical_value * se
    upper = mean + critical_value * se
    return lower, upper 

In [49]:
confidence_interval_a = confidence_interval(mean_a, std_a, critical_a)
confidence_interval_b = confidence_interval(mean_b, std_b, critical_b)


print(f'Group A: {np.round(confidence_interval_a, 3)}')
print('Group B: {:.4}, {:.4}'.format(*confidence_interval_b))

Group A: [3.049 3.7  ]
Group B: 3.073, 3.708
